In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
from mprompt.modules.emb_diff_module import EmbDiffModule
import numpy as np
import matplotlib
import imodelsx.util
import re
import mprompt.viz
import scipy.special
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from mprompt.methods.m4_evaluate import D5_Validator
import openai
openai.api_key_path = os.path.expanduser('~/.OPENAI_KEY')

def moving_average(a, n=3):
    assert n % 2 == 1, 'n should be odd'
    diff = n // 2
    vals = []
    # calculate moving average in a window 2
    # (1, 4)
    for i in range(diff, len(a) + diff):
        l = i - diff
        r = i + diff + 1
        vals.append(np.mean(a[l: r]))
    return np.nan_to_num(vals)
nlp = English()
nlp.tokenizer = Tokenizer(nlp.vocab, token_match=re.compile(r'\S+').match) # only split on whitespace

# Get prompts

In [18]:
expls = [
    'baseball',
    'animals',
    'water',
    'movement',
    'religion',
    'technology',
    'time'
]

In [19]:
prompt_init = 'Write the beginning paragraph of a story about "{expl}". Make sure it contains several references to "{expl}".'
prompt_continue = 'Write the next paragraph of the story, but now make it about "{expl}". Make sure it contains several references to "{expl}".'
prompts = [prompt_init.format(expl=expls[0])] + [prompt_continue.format(expl=expl) for expl in expls[1:]]
for p in prompts:
    print(p)

Write the beginning paragraph of a story about "baseball". Make sure it contains several references to "baseball".
Write the next paragraph of the story, but now make it about "animals". Make sure it contains several references to "animals".
Write the next paragraph of the story, but now make it about "water". Make sure it contains several references to "water".
Write the next paragraph of the story, but now make it about "movement". Make sure it contains several references to "movement".
Write the next paragraph of the story, but now make it about "religion". Make sure it contains several references to "religion".
Write the next paragraph of the story, but now make it about "technology". Make sure it contains several references to "technology".
Write the next paragraph of the story, but now make it about "time". Make sure it contains several references to "time".


# Get paragraphs

In [55]:
def get_paragraphs(
        prompts,
        checkpoint='gpt-3.5-turbo',
        PROMPT_FIRST_PREFIX='Write the next paragraph of the story, but now make it about',
        PROMPT_NEXT_PREFIX='Write the beginning paragraph of a story about'
):
    # example messages
    # [
    #   {'role': 'system', 'content': 'You are a helpful assistant.'},
    #   {'role': 'user', 'content': 'Write the beginning paragraph of a story about "baseball". Make sure it contains several references to "baseball".'},
    #   {'role': 'assistant', 'content': 'The crack of the bat echoed through the stadium as the ball soared over the outfield fence. The crowd erupted into cheers, their excitement palpable. It was a beautiful day for baseball, with the sun shining down on the field and the smell of freshly cut grass filling the air. The players on the field were focused and determined, each one ready to give their all for their team. Baseball was more than just a game to them; it was a passion, a way of life. And as they took their positions on the field, they knew that anything was possible in this great game of baseball.'},
    #   {'role': 'user', 'content': 'Write the next paragraph of the story, but now make it about "animals". Make sure it contains several references to "animals".'},
    # ]

    llm = mprompt.llm.get_llm(checkpoint)
    response = None
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    all_content = []
    for i in range(len(prompts)):
        messages.append({"role": "user", "content": prompts[i]})
        all_content.append(messages[-1])
        # for message in messages:
            # print(message)
        response = llm(messages)

        if response is not None:
            response_text = response['choices'][0]['message']['content']
            messages.append({"role": "assistant", "content": response_text})
            all_content.append(messages[-1])

        # need to drop beginning of story whenever we approach the tok limit
        # gpt-3.5.turbo has a limit of 4096, and it cant generate beyond that
        num_tokens = response['usage']['total_tokens']
        # print('num_tokens', num_tokens)
        if num_tokens >= 3200:
            # drop the first (assistant, user) pair in messages
            messages = [messages[0]] + messages[3:]

            # rewrite the original prompt to now say beginning paragraph rather than next paragraph
            messages[1]['content'] = messages[1]['content'].replace(PROMPT_FIRST_PREFIX, PROMPT_NEXT_PREFIX)
    
    # extract out paragraphs
    paragraphs = [d['content'] for d in all_content if d['role'] == 'assistant']
    paragraphs
    assert len(paragraphs) == len(prompts)
    return paragraphs
paragraphs = get_paragraphs(prompts)

cached!
cached!
cached!
cached!
cached!
cached!
cached!


In [57]:
for para in paragraphs:
    print(para)

The crack of the bat echoed through the stadium as the ball soared over the outfield fence. The crowd erupted into cheers, their excitement palpable. It was a beautiful day for baseball, with the sun shining down on the field and the smell of freshly cut grass filling the air. The players on the field were focused and determined, each one ready to give their all for their team. Baseball was more than just a game to them; it was a passion, a way of life. And as they took their positions on the field, they knew that anything was possible in this great game of baseball.
As the game continued, a group of animals gathered outside the stadium, drawn by the sounds and smells of the game. A family of raccoons peered through a hole in the fence, their curious eyes watching as the players ran back and forth across the field. A pack of stray dogs barked excitedly, tails wagging as they caught whiffs of hot dogs and popcorn. Even a few birds had flown in to catch a glimpse of the action, perching 

# Visualize data heatmap

### Get embedding dists

In [ ]:
# mod = EmbDiffModule()
val = D5_Validator()

In [ ]:
story_running = ''
scores = {}
for i in range(len(expls)):
# for i in range(1):
    expl = expls[i].lower()
    text = paragraphs[i]
    words = text.split()
    prompt = prompts[i]

    ngrams = imodelsx.util.generate_ngrams_list(text.lower(), ngrams=3, tokenizer_ngrams=nlp.tokenizer)
    ngrams = [words[0], words[0] + ' ' + words[1]] + ngrams

    # # embdiff-based viz
    # mod._init_task(expl)    
    # neg_dists = mod(ngrams)
    # assert len(ngrams) == len(words) == len(neg_dists)
    # # neg_dists = scipy.special.softmax(neg_dists)
    # plt.plot(neg_dists)
    # plt.plot(moving_average(neg_dists, n=5))
    # neg_dists = moving_average(neg_dists, n=3)
    # neg_dists = (neg_dists - neg_dists.min()) / (neg_dists.max() - neg_dists.min())
    # neg_dists = neg_dists / 2 + 0.5 # shift to 0.5-1 range
    # s = mprompt.viz.colorize(words, neg_dists, title=expl, subtitle=prompt)

    # validator-based viz
    probs = np.array(val.validate_w_scores(expl, ngrams))
    probs_disp = moving_average(probs, n=3)
    probs_disp = probs_disp / 2 + 0.5 # shift to 0.5-1 range
    s = mprompt.viz.colorize(words, probs_disp, title=expl, subtitle=prompt)
    
    # viz
    display(HTML(s))
    story_running += ' ' + s

with open('../results/story_running.html', 'w') as f:
    f.write(story_running)

# Quantify synthetic data
Calculate mean match for each paragraph to each explanation

In [ ]:
# story_running = ''
n = len(expls)
scores = np.zeros((n, n))
for i in tqdm(range(n)):
    expl = expls[i]
    for j in range(n):
        text = paragraphs[j].lower()
        words = text.split()

        ngrams = imodelsx.util.generate_ngrams_list(text, ngrams=3, tokenizer_ngrams=nlp.tokenizer)
        ngrams = [words[0], words[0] + ' ' + words[1]] + ngrams

        # validator-based viz
        probs = np.array(val.validate_w_scores(expl, ngrams)) > 0.5
        scores[i, j] = probs.mean()

In [ ]:
s = scores
# s = scipy.special.softmax(scores, axis=1)
# s = (s - s.min()) / (s.max() - s.min())
plt.figure(figsize=(6, 5))
plt.imshow(s)
plt.xticks(range(n), expls, rotation=90)
plt.yticks(range(n), expls)
plt.ylabel('Explanation for generation')
plt.xlabel('Explanation for matching')
plt.colorbar(label='Fraction of matching ngrams')
plt.tight_layout()
plt.show()